In [14]:
!pip install elasticsearch -q
!pip install langchain_community -q
!pip install langchain_elasticsearch -q
!pip install -U sentence-transformers -q

In [8]:
### REF: https://python.langchain.com/v0.1/docs/integrations/vectorstores/elasticsearch/
import elasticsearch
## from langchain_community.vectorstores import ElasticVectorSearch,ElasticsearchStore
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_elasticsearch import ElasticsearchStore
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [10]:
es_client= elasticsearch.Elasticsearch(
    hosts=["https://spectra-poweredge:9200"],
    basic_auth=('elastic', 'elastic'),
    max_retries=10,
    verify_certs=False
)

In [15]:
model_name = "sentence-transformers/all-mpnet-base-v2"
index_name = "vulnerabilities-data"
hf = HuggingFaceEmbeddings(model_name=model_name)
db = ElasticsearchStore(embedding=hf, es_connection=es_client, index_name=index_name)

/usr/local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [17]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import CharacterTextSplitter

##loader = TextLoader("./state_of_the_union.txt")
##documents = loader.load()
loader = CSVLoader(file_path="./cve.csv")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1220, which is longer than the specified 500
Created a chunk of size 1215, which is longer than the specified 500
Created a chunk of size 1211, which is longer than the specified 500
Created a chunk of size 1231, which is longer than the specified 500
Created a chunk of size 1219, which is longer than the specified 500
Created a chunk of size 1233, which is longer than the specified 500
Created a chunk of size 1037, which is longer than the specified 500
Created a chunk of size 562, which is longer than the specified 500
Created a chunk of size 502, which is longer than the specified 500
Created a chunk of size 715, which is longer than the specified 500
Created a chunk of size 715, which is longer than the specified 500
Created a chunk of size 715, which is longer than the specified 500
Created a chunk of size 715, which is longer than the specified 500
Created a chunk of size 650, which is longer than the specified 500
Created a chunk of size 640, which is lon

In [ ]:
db = ElasticsearchStore.from_documents(
    docs,
    hf,
    es_connection=es_client,
    index_name=index_name,
)

db.client.indices.refresh(index=index_name)

In [24]:
es_client.indices.get_alias(index=index_name+"*")

ObjectApiResponse({'llm-basic': {'aliases': {}}})

In [28]:
query = "What did the president say about Ketanji Brown Jackson"
results = db.similarity_search(query)
results[0].page_content

'One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'